# WRC Scraper  2020


*TO DO  - add in database elements. Schema will be in `wrcResults2020.sql`.*

There may be duplication of data across various objects in the final class. This is for convenience as much as anything, trying to preserve both captures of the original data with combined or derived data, as well as trying to put the data in places we can easily find it. Consistency in a live setting is where this may fall apart!

*Add in `asyncio` scheduler elements to call WRC API regularly. Avoid race conditions by scheduling items together in the same scheduled event if they compete for the same write resource.*

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext pycodestyle_magic
%flake8_on --ignore D100

In [2]:
from WRC_2020_scraper import *
from WRC_2020_scraper import getSeasonCategories

1:1: F403 'from WRC_2020_scraper import *' used; unable to detect undefined names


In [3]:
import pandas as pd
from sqlite_utils import Database
import uuid
import pickle

In [4]:
TESTDB = 'testdbfy.db'
!rm $TESTDB

1: E999 SyntaxError: invalid syntax


In [5]:
from WRCUtils2020 import _isnull, _notnull, _checkattr, _jsInt, listify

In [6]:
# TO DO - define a class for each table
import warnings

#https://stackoverflow.com/questions/28208949/log-stack-trace-for-python-warning/29567225
import traceback

_formatwarning = warnings.formatwarning

4:1: E265 block comment should start with '# '
4:80: E501 line too long (89 > 79 characters)


The following generates stacktrace warnings useful when debugging in IPython notebook kernel environment.

In [7]:
def formatwarning_tb(*args, **kwargs):
    """Format warning message with added stack trace."""
    s = _formatwarning(*args, **kwargs)

    _tb = traceback.format_stack()
    tb = []
    for i in _tb:
        if 'ipython-input' in i:
            tb.append(i)
    s += ''.join(tb[:-1])
    return s


warnings.formatwarning = formatwarning_tb
# logging.captureWarnings(True)


class IPythonWarner:
    """Tools for reporting warnings."""

    def __init__(self, nowarn=False):
        """Initialise class with nowarn attribute."""
        self.nowarn = nowarn or None

    def warner(self, cond, msg, nowarn=None):
        """Test a condition and if True display a warning message."""
        if nowarn is None:
            nowarn = self.nowarn

        if not nowarn and cond:
            warnings.warn(msg)


In [8]:
class Pickler:
    """Pickle support for class."""

    def __init__(self, nowarn=False):
        """Initialise pickler."""
        pass

    def pickle(self, fn=None):
        """Pickle self."""
        fn = fn or f'{uuid.uuid4().hex}.pickle'
        self.db = None
        with open(fn, 'wb') as f:
            pickle.dump(self, f)
            print(f'Pickled to {fn}')
        self.db_connect(dbname=zz.dbname)


In [9]:
# TO DO

# how about a SqliteDB class that can provde db methods. Use sqlite_utils.
# upserter is most important, so can save each table: upsert(table, df, key)
# upserter may require col renaming or additional cols to df before upsert

3:1: E265 block comment should start with '# '


In [10]:
class SQLiteDB:
    """Simple sqlite utils."""

    def __init__(self, dbname=None, nowarn=True):
        """Create db connection if dbname provided."""
        if dbname:
            self.db_connect(dbname)

    def db_connect(self, dbname=None):
        """
        Connect to a db.

        Give the file a randomly generated name if one not provided.
        """
        dbname = dbname or f'{uuid.uuid4().hex}.db'
        if not _checkattr(self, dbname) or (dbname and self.dbname != dbname):
            self.dbname = dbname
        else:
            self.dbname = dbname

        self.db = Database(self.dbname)

    def upsert(self, table, data=None, pk=None):
        """
        Upsert data to database.

        If we forget to create a database, this will create one.
        """
        print('upserting')
        _upserts = []

        if isinstance(table, str):
            if _notnull(data) and pk is not None:
                # One table
                _upserts.append((table, data, pk))
            # Check data is None. It may be {}, which would be data...
            elif hasattr(self, table) and data is None and pk is not None:
                # One table, data from self attribute
                data = getattr(self, table)
                _upserts.append((table, data, pk))
        elif isinstance(table, tuple) and len(table) == 2 and _isnull(data):
            (_table, _pk) = table
            if isinstance(_table, str) and hasattr(self, _table) and _pk is not None:
                _data = getattr(self, _table)
                _upserts.append((_table, _data, _pk))
        elif isinstance(table, list) and _isnull(data):
            # Several tables from self  attributes: [(table, pk), ..]
            for _t in table:
                if isinstance(_t, tuple) and len(_t) == 2:
                    (_table, _pk) = _t
                    if isinstance(_table, str) and hasattr(self, _table) and _pk is not None:
                        _data = getattr(self, _table)
                        _upserts.append((_table, _data, _pk))

        # TO DO - if the pk is none, and data is a df, use the df index?

        if not _checkattr(self, 'dbname') and not _checkattr(self, 'db'):
            self.db_connect()

        # The alter=True allows us the table to be modified if we have extra columns
        for (_table, _data, _pk) in _upserts:
            # TO DO - the try is to cope with this issue:
            # https://github.com/simonw/sqlite-utils/issues/73#issuecomment-571138093
            if isinstance(_data, pd.DataFrame) and _notnull(_data) and _pk is not None:
                try:
                    self.db[_table].upsert_all(_data.to_dict(orient='records'),
                                               pk=_pk, alter=True)
                except:
                    try:
                        self.db[_table].insert_all(_data.to_dict(orient='records'),
                                                   pk=_pk, alter=True, replace=True)
                    except:
                        warning(f"Couldn't add data to {_table} with PK {_pk}")

            elif isinstance(_data, dict) and _data and _pk is not None:
                try:
                    self.db[_table].upsert_all(_data, pk=_pk, alter=True)
                except:
                    try:
                        self.db[_table].insert_all(_data, pk=_pk,
                                                   alter=True, replace=True)
                    except:
                        warning(f"Couldn't add data to {_table} with PK {_pk}")
                        
        # TO DO  - what if we have a dict of dicts like for the stagesplits etc?

    def dbfy(self, table, data=None, pk=None):
        """If we have a database, use it."""
        if hasattr(self, 'db'):
            self.upsert(table, data, pk)

    def q(self, query):
        """Run SQL query on db, returning a dataframe."""
        if hasattr(self, 'db'):
            return pd.read_sql(query, self.db.conn)

    def dbtables(self):
        """Show tables in db."""
        query = "SELECT * FROM sqlite_master where type='table';"
        return self.q(query)

43:80: E501 line too long (85 > 79 characters)
51:80: E501 line too long (93 > 79 characters)
60:80: E501 line too long (84 > 79 characters)
64:80: E501 line too long (87 > 79 characters)
68:17: E722 do not use bare 'except'


Error in callback <bound method VarWatcher.auto_run_flake8 of <pycodestyle_magic.VarWatcher object at 0x109c02710>> (for post_run_cell):


ValueError: invalid literal for int() with base 10: '`, it also catches unexpected events like memory errors, interrupts, system exit, and so on.  Prefer `except Exception'

In [11]:
class WRCBase(IPythonWarner, SQLiteDB, Pickler):
    """Base class for all WRC stuff."""

    def __init__(self, nowarn=True, dbname=None):
        """Add universal classes to WRCBase class."""
        IPythonWarner.__init__(self, nowarn=nowarn)
        SQLiteDB.__init__(self, dbname=dbname)

    def _null():
        pass

In [12]:
class WRCSeasonBase(WRCBase):
    """Base class for things related to with seasons."""

    def __init__(self, season_external_id=None, autoseed=False,
                 nowarn=True, dbname=None):
        """Add WRCBase class. Autoseed with season details if required."""
        WRCBase.__init__(self, nowarn=nowarn, dbname=dbname)

        self.season_external_id = _jsInt(season_external_id)
        if not self.season_external_id and autoseed:
            self._check_season_external_id()

    def _check_season_external_id(self, season_external_id=None):
        """Check that season_external_id exists and if not, get one."""
        self.season_external_id = _jsInt(season_external_id) or self.season_external_id
        if not _checkattr(self, 'season_external_id'):
            self.fetchSeasonExternalId(season_external_id)

    def fetchSeasonExternalId(self, season_external_id=None):
        """Fetch a season external ID howsoever we can."""
        # Get current one from active rally
        # It's also available from current_season_events
        event, days, channels = getActiveRally()
        self.event, self.days, self.channels = event, days, channels

        # The returned np.int64 is not JSON serialisable
        self.season_external_id = _jsInt(event.loc[0, 'season.externalId'])
        # Upsert the data if there's a db connection
        self.dbfy([('event', 'id'),
                   ('days', 'id'),
                   ('channels', 'id')])

15:80: E501 line too long (87 > 79 characters)
19:10: N802 function name 'fetchSeasonExternalId' should be lowercase


In [13]:
zz = WRCSeasonBase(autoseed=True)  # .season_external_id
zz.db_connect()

In [14]:
zz.pickle()
!ls *.pickle

Pickled to 78cef01c9e1e4c79902515251109520e.pickle
565f454998be4a6cba2782b07ffd53f9.pickle b6eaaf025e7f4e3d8873d23b4161c0cd.pickle
6b9684d20d8f4280924a2e603c395bd0.pickle c17aa75f57c645d8aab909b865c9a340.pickle
78cef01c9e1e4c79902515251109520e.pickle c43ec2859c204e5484b47babeb115ea2.pickle
92b0b4a451ad4cd78f328d765ab8b729.pickle wrc-montecarlo.pickle
adc2895325584b929614ae4110ac14f7.pickle


1: E999 SyntaxError: invalid syntax
2:6: E225 missing whitespace around operator


In [15]:
class WRCSeasonCategories(WRCSeasonBase):
    """Class describing season categories."""

    def __init__(self, autoseed=False, nowarn=True, dbname=None):
        """
        Add season categories to base class.

        Autoseed if required. Update db if available.
        """
        WRCSeasonBase.__init__(self, nowarn=nowarn, dbname=dbname)

        self.season_categories = None
        self.championship_codes = None

        if autoseed:
            self.fetchSeasonCategories()

    def fetchSeasonCategories(self):
        """Fetch season categories from WRC API. Add to db if required."""
        self.season_categories = getSeasonCategories()
        self.championship_codes = getSeasonChampionshipCodes()
        self.dbfy([('season_categories', 'id'),
                   ('championship_codes', 'id')])

18:10: N802 function name 'fetchSeasonCategories' should be lowercase


In [16]:
zz = WRCSeasonCategories(dbname=TESTDB)
zz.fetchSeasonCategories()
zz.championship_codes
zz.dbtables()

upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...


In [17]:
class WRCChampionship(WRCSeasonCategories):
    """Class for championship."""

    def __init__(self, category='WRC', typ='drivers',
                 season_external_id=None,
                 autoseed=False, nowarn=True, dbname=None):
        """Build on season categories with championship details."""
        WRCSeasonCategories.__init__(self, autoseed=False,
                                     nowarn=True, dbname=dbname)

        self.championships = {}

        if autoseed:
            fetchChampionship(self, category=category, typ=typ,
                              season_external_id=season_external_id)

    def fetchChampionship(self, category='WRC',
                          typ='drivers', season_external_id=None):
        """Get championship data from WRC API and update db if available."""
        self._check_season_external_id(season_external_id)
        _c = self.championships
        if category not in _c:
            _c[category] = {}
        _cc = _c[category]
        if typ not in _cc:
            _cc[typ] = {}
        _cct = _cc[typ]
        (_cct['championship'], _cct['championshipRounds'],
         _cct['championshipEntries']) = getChampionship(category=category,
                                                        typ=typ, season_external_id=season_external_id)

        self.dbfy('championship', _cct['championship'], 'championshipId')
        self.dbfy('championshipRounds', _cct['championshipRounds'],
                  ['eventId', 'championshipId'])
        self.dbfy('championshipEntries', _cct['championshipEntries'],
                  ['championshipEntryId', 'championshipId', 'personId'])

17:10: N802 function name 'fetchChampionship' should be lowercase
30:80: E501 line too long (103 > 79 characters)


In [18]:
zz = WRCChampionship(dbname=TESTDB)
zz.fetchChampionship()
zz.dbtables()

upserting
upserting
upserting
upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...


In [19]:
class WRCChampionshipStandings(WRCChampionship):
    """Standings for a championship."""

    def __init__(self, category='WRC', typ='drivers',
                 season_external_id=None, autoseed=False,
                 nowarn=True, dbname=None):
        """Build on championship with championship standings."""
        WRCChampionship.__init__(self, autoseed=False,
                                 season_external_id=season_external_id,
                                 nowarn=True, dbname=dbname)

        self.championship_standings = {}

        if autoseed:
            fetchChampionshipStandings(self, category=category, typ=typ,
                                       season_external_id=season_external_id)

    def fetchChampionshipStandings(self, category='WRC',
                                   typ='drivers', season_external_id=None):
        """Get championship standings from API and update db if available."""
        self._check_season_external_id(season_external_id)
        _c = self.championship_standings
        if category not in _c:
            _c[category] = {}
        _cc = _c[category]
        if typ not in _cc:
            _cc[typ] = {}
        _cct = _cc[typ]
        (_cct['championship_standings'],
         _cct['round_results']) = getChampionshipStandings(category=category,
                                                           typ=typ,
                                                           season_external_id=season_external_id)

        self.dbfy('championship_standings', _cct['championship_standings'],
                  ['championshipId', 'championshipEntryId'])
        self.dbfy('round_results', _cct['round_results'],
                  ['championshipEntryId', 'championshipId', 'eventId'])


18:10: N802 function name 'fetchChampionshipStandings' should be lowercase
32:80: E501 line too long (97 > 79 characters)


In [20]:
zz = WRCChampionshipStandings(dbname=TESTDB)
zz.fetchChampionshipStandings('JWRC')
zz.fetchChampionshipStandings('WRC')
zz.championship_standings
zz.dbtables()

upserting
upserting
upserting
upserting
upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


TO DO  - in a clea environment, the following fails on first run. But if we run it again, it works.

In [21]:
# TO DO - need a more general season events class?
# If, that is, we can we look up arbitrary season events...


class WRCActiveSeasonEvents(SQLiteDB):
    """Class for Season events."""

    def __init__(self, autoseed=False, dbname=None):
        """Create active season event class with db support."""
        SQLiteDB.__init__(self, dbname=dbname)

        if autoseed:
            self.fetchActiveSeasonEvents()

    def fetchActiveSeasonEvents(self):
        """Fetch active season data from WRC API."""
        (self.current_season_events, self.eventdays,
         self.eventchannel) = getActiveSeasonEvents()
        self.dbfy([('current_season_events', 'id'),
                   ('eventdays', 'id'),
                   ('eventchannel', 'id')])

15:10: N802 function name 'fetchActiveSeasonEvents' should be lowercase


In [22]:
zz = WRCActiveSeasonEvents(autoseed=True, dbname=TESTDB)
zz.current_season_events.head()
zz.fetchActiveSeasonEvents()
zz.dbtables()

upserting
upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [23]:
zz.dbfy([('current_season_events', 'id')])
zz.dbtables()

upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [24]:
class WRCRally_sdb(WRCBase):
    """
    Base class for things with an sdbRallyId.

    Can also help find an active sdbRallyId.
    """

    def __init__(self, sdbRallyId=None,
                 autoseed=False, nowarn=True, dbname=None):
        """Initialise things that have a rallyId."""
        WRCBase.__init__(self, nowarn=nowarn, dbname=dbname)

        self.warner(not sdbRallyId, "sdbRallyId should really be set...")

        self.sdbRallyId = _jsInt(sdbRallyId)

        if autoseed:
            self._checkRallyId(sdbRallyId)

    def _checkRallyId(self, sdbRallyId=None):
        """Return a rally ID or lookup active one."""
        sdbRallyId = _jsInt(sdbRallyId) or self.sdbRallyId
        if not _checkattr(self, 'sdbRallyId'):
            self.activerally = WRCActiveRally()
            self.sdbRallyId = self.activerally.sdbRallyId
            self.name = self.activerally.name

        return self.sdbRallyId

1:8: N801 class name 'WRCRally_sdb' should use CapWords convention
8:25: N803 argument name 'sdbRallyId' should be lowercase
20:10: N802 function name '_checkRallyId' should be lowercase
20:30: N803 argument name 'sdbRallyId' should be lowercase
22:10: N806 variable 'sdbRallyId' in function should be lowercase


In [25]:
zz = WRCRally_sdb(nowarn=False, dbname=TESTDB)#.nowarn
zz.dbtables()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: sdbRallyId should really be set...
  File "<ipython-input-25-96f7890a458b>", line 1, in <module>
    zz = WRCRally_sdb(nowarn=False, dbname=TESTDB)#.nowarn
  File "<ipython-input-24-22fef7bff9ce>", line 13, in __init__
    self.warner(not sdbRallyId, "sdbRallyId should really be set...")
  File "<ipython-input-7-2f91cd85b556>", line 31, in warner
    warnings.warn(msg)


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


1:47: E261 at least two spaces before inline comment
1:47: E262 inline comment should start with '# '


In [26]:
class WRCLive(WRCBase):
    """Base class for live rallies."""

    def __init__(self, live=False, dbname=None):
        """Initialise class for live datasets."""
        WRCBase.__init__(self, dbname=dbname)

        self.live = live

In [27]:
class WRCActiveRally(WRCRally_sdb, WRCLive):
    """Create class for the active rally."""

    def __init__(self, live=False, dbname=False):
        """Active rally class builds on Rallyid and live classes."""
        WRCRally_sdb.__init__(self, nowarn=True, dbname=dbname)
        WRCLive.__init__(self, live=live)

        self.fetchData()

    def fetchData(self):
        """Get data for active rally."""
        event, days, channels = getActiveRally()
        self.event, self.days, self.channels = event, days, channels

        self.dbfy([('event', 'id'),
                   ('days', 'id'),
                   ('channels', 'id')])

        # np.int64 is not JSON serialisable
        self.sdbRallyId = int(event.loc[0, 'id'])

        self.name = event.loc[0, 'name']

11:10: N802 function name 'fetchData' should be lowercase


In [28]:
# zz = WRCActiveRally(dbname=TESTDB)#.sdbRallyId
# zz.dbtables()
zz = WRCActiveRally()

In [29]:
zz.event

,id,name,externalIdRally,externalIdEvent,timezone,active,countdown,jwrc,images.format16x9.320x180,images.format16x9.160x90,...,winner.nation.alpha2,winner.nation.alpha3,winner.nation.ioc,winner.birthDate,winner.birthPlace,winner.debutDate,winner.debutPlace,winner.website,winner.externalId,winner.page
0,100,Rallye Monte Carlo,153,124,1,True,False,False,https://www.wrc.com/images/redaktion/Web-2020/...,https://www.wrc.com/images/redaktion/Web-2020/...,...,BE,BEL,BEL,1988-06-16,Belgium,2009-01-12,Rally de Portugal,https://www.thierryneuville.com/,762,None


In [30]:
zz = WRCRally_sdb(autoseed=True)
print(zz.sdbRallyId)

100


We use the `.fetchData()` method so as to ry not to be greedy. This way, we can define a class and start to work towards only grabbling the data if we need it.

In [31]:
class WRCRetirements(WRCRally_sdb):
    """Callable class for retirements."""

    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, dbname=None):
        """Initialise retirements class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              autoseed=autoseed, dbname=dbname)

        self.retirements = None

        if self.sdbRallyId:
            self.fetchRetirements(self.sdbRallyId)

    def fetchRetirements(self, sdbRallyId=None):
        """Fetch retirements data from WRC API. Add to db if available."""
        self._checkRallyId(sdbRallyId)
        self.retirements = getRetirements(self.sdbRallyId)

        # Upsert the data if there's a db connection
        self.dbfy([('retirements', 'retirementId')])

    def __call__(self):
        """Make retirements class callable. Returns dataframe."""
        return self.retirements

4:25: N803 argument name 'sdbRallyId' should be lowercase
15:10: N802 function name 'fetchRetirements' should be lowercase
15:33: N803 argument name 'sdbRallyId' should be lowercase


In [32]:
zz = WRCRetirements(autoseed=True, dbname=TESTDB)
display(zz.retirements.head(3))
zz.dbtables()

upserting


,retirementId,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,status
0,1475,6591,20710,OFF ROAD,2020-01-23T20:40:00Z,0001-01-01T00:00:00+00:00,Temporary
1,1476,6588,20687,MECHANICAL,2020-01-23T21:57:00Z,0001-01-01T00:00:00+00:00,Temporary
2,1477,6591,20750,OFF ROAD,2020-01-23T22:51:00Z,0001-01-01T00:00:00+00:00,Permanent


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [33]:
class WRCPenalties(WRCRally_sdb):
    """Callable class for penalties."""

    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, dbname=None):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              autoseed=autoseed, dbname=dbname)

        self.penalties = None

        if self.sdbRallyId:
            self.fetchPenalties(self.sdbRallyId)

    def fetchPenalties(self, sdbRallyId=None):
        """Fetch penalties data from WRC API. Add to db if available."""
        self._checkRallyId(sdbRallyId)
        self.penalties = getPenalties(self.sdbRallyId)

        # Upsert the data if there's a db connection
        self.dbfy([('penalties', 'penaltyId')])

    def __call__(self):
        """Make penalties class callable. Returns dataframe."""
        return self.penalties

4:25: N803 argument name 'sdbRallyId' should be lowercase
15:10: N802 function name 'fetchPenalties' should be lowercase
15:31: N803 argument name 'sdbRallyId' should be lowercase


In [34]:
zz = WRCPenalties(autoseed=True, dbname=TESTDB)
display(zz.penalties.head(3))
zz.dbtables()

upserting


,penaltyId,controlId,entryId,penaltyDurationMs,penaltyDuration,reason
0,725,6592,20730,10000,PT10S,FALSE START
1,726,6592,20753,10000,PT10S,FALSE START
2,727,6590,20760,10000,PT10S,1 MIN LATE


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [35]:
class WRCStagewinners(WRCRally_sdb):
    """Callable class for penalties."""

    def __init__(self, sdbRallyId=None, live=False, autoseed=False, dbname=None):
        """Initialise penalties class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              autoseed=autoseed, dbname=dbname)

        self.stagewinners = None

        if self.sdbRallyId:
            self.fetchStagewinners(self.sdbRallyId)

    def fetchStagewinners(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self.stagewinners = getStagewinners(self.sdbRallyId)

        self.dbfy(('stagewinners', ['stageId', 'entryId']))

    def __call__(self):
        """Make stagewinners class callable. Returns dataframe."""
        return self.stagewinners

4:25: N803 argument name 'sdbRallyId' should be lowercase
4:80: E501 line too long (81 > 79 characters)
14:10: N802 function name 'fetchStagewinners' should be lowercase
14:34: N803 argument name 'sdbRallyId' should be lowercase


In [36]:
zz = WRCStagewinners(dbname=TESTDB)
zz.fetchStagewinners()
display(zz.stagewinners.head())
zz.dbtables()

upserting


,stageId,entryId,stageName,elapsedDurationMs,elapsedDuration
0,1538,20684,Bayons - Bréziers,983700,00:16:23.7000000
1,1528,20685,Malijai - Puimichel (Live TV),593400,00:09:53.4000000
2,1533,20686,Curbans - Venterol 1,802000,00:13:22
3,1534,20686,Saint-Clément - Freissinières 1,703300,00:11:43.3000000
4,1535,20686,Avançon - Notre-Dame-du-Laus 1,780700,00:13:00.7000000


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [37]:
class WRCItinerary(WRCRally_sdb, WRCLive):
    """Class for WRC2020 Itinerary."""

    def __init__(self, sdbRallyId=None, live=False, autoseed=False, dbname=None):
        """Initialise itinerary class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId, autoseed=autoseed, dbname=dbname)
        WRCLive.__init__(self, live=live)

        self.itinerary = None
        self.legs = None
        self.sections = None
        self.controls = None
        self.stages = None

        if self.sdbRallyId:
            self.fetchItinerary(sdbRallyId)

    def _checkItinerary(self):
        """
        Check itinerary.

        If rally not known, use active rally.
        Also set a default startListId.
        """
        _itinerary_items = ['itinerary', 'legs', 'sections', 'controls', 'stages']
        if not all([_checkattr(self, i) for i in _itinerary_items]):
            self.fetchItinerary()

    def fetchItinerary(self, sdbRallyId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        (self.itinerary, self.legs, self.sections,
         self.controls, self.stages) = getItinerary(self.sdbRallyId)

        _ccols = ['code'] + (list(set(self.controls.columns) - set(self.stages.columns)))
        self.richstages = self.stages.merge(self.controls[_ccols], on='code')

        self.dbfy([('itinerary', ['itineraryId']),
                   ('legs', ['itineraryLegId']),
                   ('sections', ['itinerarySectionId']),
                   ('controls', ['controlId']),
                   ('stages', ['stageId']),
                   ('richstages', ['stageId'])])

    def getStageIdFromCode(self, code=None):
        """Return a stageID from a single codes."""
        if code and isinstance(code, str) and _checkattr(self, 'stages'):
            _df = self.stages[self.stages['code'] == code][['rallyid', 'stageId']]
            return tuple(_df.iloc[0])
        elif code and isinstance(code, list):
            # TO DO - this might be dangerous if we are expencting a single tuple response
            getStageIdsFromCode(code, response='tuples')

    def getStageIdsFromCode(self, code=None, response='dict'):
        """Return a stageID from one or more codes."""
        if code and _checkattr(self, 'stages'):
            _df = self.stages[self.stages['code'].isin(listify(code))][['code', 'rallyid', 'stageId']]
            if response == 'df':
                return _df
            elif response == 'tuples':
                return list(_df.itertuples(index=False, name=None))
            return _df.set_index('code').to_dict(orient='index')

    def getStageIds(self, typ='all', response='dict'):
        """Return stageIDs by stage status."""
        if _checkattr(self, 'stages'):
            if typ == 'all':
                _df = self.stages[['code', 'rallyid', 'stageId']]
            else:
                if typ == 'completed':
                    _statuses = ['Completed', 'Interrupted']
                elif typ == 'onlycompleted':
                    _statuses = ['Completed']
                elif typ == 'interrupted':
                    _statuses = ['Interrupted']
                _df = self.stages[self.stages['status'].isin(_statuses)][['code', 'rallyid', 'stageId']]

            if response == 'df':
                return _df
            elif response == 'tuples':
                return list(_df.itertuples(index=False, name=None))
            return _df.set_index('code').to_dict(orient='index')


4:25: N803 argument name 'sdbRallyId' should be lowercase
4:80: E501 line too long (81 > 79 characters)
6:80: E501 line too long (92 > 79 characters)
18:10: N802 function name '_checkItinerary' should be lowercase
25:80: E501 line too long (82 > 79 characters)
29:10: N802 function name 'fetchItinerary' should be lowercase
29:31: N803 argument name 'sdbRallyId' should be lowercase
35:80: E501 line too long (89 > 79 characters)
45:10: N802 function name 'getStageIdFromCode' should be lowercase
48:80: E501 line too long (82 > 79 characters)
51:80: E501 line too long (90 > 79 characters)
54:10: N802 function name 'getStageIdsFromCode' should be lowercase
57:80: E501 line too long (102 > 79 characters)
64:10: N802 function name 'getStageIds' should be lowercase
76:80: E501 line too long (104 > 79 characters)


In [38]:
sdbRallyId = 100
zz = WRCItinerary(sdbRallyId, autoseed=True, dbname=TESTDB)
zz.dbtables()

upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


1:2: N816 variable 'sdbRallyId' in global scope should not be mixedCase


In [39]:
zz.getStageIds('interrupted')
zz.getStageIds()

{'SS1': {'rallyid': 100, 'stageId': 1528},
 'SS2': {'rallyid': 100, 'stageId': 1538},
 'SS3': {'rallyid': 100, 'stageId': 1533},
 'SS4': {'rallyid': 100, 'stageId': 1534},
 'SS5': {'rallyid': 100, 'stageId': 1535},
 'SS6': {'rallyid': 100, 'stageId': 1539},
 'SS7': {'rallyid': 100, 'stageId': 1537},
 'SS8': {'rallyid': 100, 'stageId': 1536},
 'SS9': {'rallyid': 100, 'stageId': 1532},
 'SS10': {'rallyid': 100, 'stageId': 1531},
 'SS11': {'rallyid': 100, 'stageId': 1530},
 'SS12': {'rallyid': 100, 'stageId': 1529},
 'SS13': {'rallyid': 100, 'stageId': 1524},
 'SS14': {'rallyid': 100, 'stageId': 1527},
 'SS15': {'rallyid': 100, 'stageId': 1525},
 'SS16': {'rallyid': 100, 'stageId': 1526}}

In [40]:
WRCItinerary(sdbRallyId=100).legs

,itineraryLegId,itineraryId,startListId,name,legDate,order,status,rallyid
0,273,240,451,Thursday 23rd January,2020-01-23,1,Completed,100
1,272,240,452,Friday 24th January,2020-01-24,2,Completed,100
2,275,240,454,Saturday 25th January,2020-01-25,3,Completed,100
3,274,240,456,Sunday 26th January,2020-01-25,4,Completed,100


In [41]:
class WRCStartlist(WRCItinerary):
    """Class for WRC2020 Startlist table."""

    def __init__(self, startListId=None, live=False, autoseed=True,
                 nowarn=False, dbname=None):
        """Startlist class, builds on itinerary class."""
        WRCItinerary.__init__(self, live=live, autoseed=autoseed,
                              dbname=dbname)

        self.startListId = _jsInt(startListId)

        self.startList = None
        self.startListItems = None

        if not nowarn and not self.startListId:
            warnings.warn("startListId should really be set..")

        if self.startListId or autoseed:
            self.fetchStartList(self.startListId)

    def _checkStartListId(self, startListId=None):
        """Return a startlistId or look one up."""
        self._checkItinerary()
        self.startListId = _jsInt(startListId or self.startListId)

        if not self.startListId:
            self.startListId = int(self.legs.loc[0, 'startListId'])

    def fetchStartList(self, startListId=None):
        """Get startlist data from API. Add to db if available."""
        self._checkStartListId(startListId)
        (startList, startListItems) = getStartlist(self.startListId)
        self.startList, self.startListItems = startList, startListItems
        self.dbfy([('startList', ['startListId']),
                   ('startListItems', ['startListItemId'])])

4:25: N803 argument name 'startListId' should be lowercase
21:10: N802 function name '_checkStartListId' should be lowercase
21:34: N803 argument name 'startListId' should be lowercase
29:10: N802 function name 'fetchStartList' should be lowercase
29:31: N803 argument name 'startListId' should be lowercase
32:11: N806 variable 'startList' in function should be lowercase
32:11: N806 variable 'startListItems' in function should be lowercase


In [42]:
zz = WRCStartlist()  # (autoseed=True, dbname=TESTDB)
# zz.fetchStartList()
zz.startListId

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: startListId should really be set..
  app.launch_new_instance()
  File "<ipython-input-42-3071abcbbc37>", line 1, in <module>
    zz = WRCStartlist()  # (autoseed=True, dbname=TESTDB)
  File "<ipython-input-41-ef42379addd3>", line 16, in __init__
    warnings.warn("startListId should really be set..")


451

In [43]:
class WRCCars(WRCRally_sdb):
    """Class for WRC2020 Cars table."""

    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, dbname=None):
        """Cars class, building on rallyid class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              autoseed=autoseed, dbname=dbname)

        self.cars = None
        self.classes = None

        if self.sdbRallyId:
            self.fetchCars(sdbRallyId)

    def fetchCars(self, sdbRallyId=None):
        """Fetch cars data from WRC API. Add to db if available."""
        self._checkRallyId(sdbRallyId)
        cars, classes = getCars(self.sdbRallyId)
        self.cars, self.classes = cars, classes
        self.dbfy([('cars', ['entryId', 'eventId']),
                   ('classes', ['entryId'])])

4:25: N803 argument name 'sdbRallyId' should be lowercase
16:10: N802 function name 'fetchCars' should be lowercase
16:26: N803 argument name 'sdbRallyId' should be lowercase


In [44]:
zz = WRCCars(autoseed=True, dbname=TESTDB)
zz.dbtables()

upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [45]:
class WRCRally(WRCRally_sdb):
    """Class for WRC2020 Rally table. This gives external ids."""

    def __init__(self, sdbRallyId=None, autoseed=False, dbname=False):
        """Rally class, builds on rallid class."""
        WRCRally_sdb.__init__(self, sdbRallyId=sdbRallyId,
                              autoseed=autoseed, dbname=dbname)

        self.rally = None
        self.eligibilities = None
        self.groups = None

        if self.sdbRallyId:
            self.fetchRally(sdbRallyId)

    def fetchRally(self, sdbRallyId=None):
        """Fetch rally details from WRC API. Add to db if avaailable."""
        self._checkRallyId(sdbRallyId)
        (self.rally, self.eligibilities, self.groups) = getRally(self.sdbRallyId)
        self.dbfy([('rally', ['sdbRallyId']),
                   ('eligibilities', ['sdbRallyId', 'category']),
                   ('groups', ['sdbRallyId', 'groupId'])])

    # TO DO - define iterators?

4:25: N803 argument name 'sdbRallyId' should be lowercase
16:10: N802 function name 'fetchRally' should be lowercase
16:27: N803 argument name 'sdbRallyId' should be lowercase
19:80: E501 line too long (81 > 79 characters)


In [46]:
zz = WRCRally(autoseed=True, dbname=TESTDB)
display(zz.rally)
# zz.dbtables()

upserting


,externalIdRally,externalIdEvent,itineraryId,name,isMain,eventClasses,sdbRallyId
0,153,124,240,WRC,True,None,100


In [47]:
# TO DO - have a check stages function to get some data...


class WRCRallyStages(WRCItinerary):
    """Class referring to all rally stages."""

    def __init__(self, sdbRallyId=None, live=False,
                 autoseed=False, nowarn=True, dbname=None):
        """Rally stages class;. Builds on itinerary."""
        WRCItinerary.__init__(self, sdbRallyId=None, live=False,
                              autoseed=autoseed, dbname=dbname)

        self.sdbRallyId = _jsInt(sdbRallyId)

        if autoseed:
            self._checkStages(self.sdbRallyId)

    def _checkStages(self, sdbRallyId=None):
        """Return a stages list or lookup list for active rally."""
        # Have we got an sdbRallyId?
        if not _checkattr(self, 'stages'):
            self.fetchStages(sdbRallyId=None)

    def fetchStages(self, sdbRallyId=None):
        """Fetch stages for a specified rally."""
        self._checkItinerary()
        self.activerally = WRCActiveRally()
        self.sdbRallyId = self.activerally.sdbRallyId
        self.name = self.activerally.name

    def lastCompletedStage(self):
        """Return last completed stage ID."""
        return self.stages[self.stages['status'] == 'Completed'].iloc[-1]['stageId']

7:25: N803 argument name 'sdbRallyId' should be lowercase
18:10: N802 function name '_checkStages' should be lowercase
18:29: N803 argument name 'sdbRallyId' should be lowercase
24:10: N802 function name 'fetchStages' should be lowercase
24:28: N803 argument name 'sdbRallyId' should be lowercase
31:10: N802 function name 'lastCompletedStage' should be lowercase


In [48]:
zz = WRCRallyStages(autoseed=True, dbname=TESTDB)
zz.richstages.head()  # stages / controls
# zz._checkStages()[1].head()
# zz.lastCompletedStage()
zz.dbtables()

upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [49]:
zz.lastCompletedStage()

1526

In [50]:
# Does this actually do anything other than possible checks?


class WRCRallyStage(WRCRallyStages):
    """
    Base class for things with a stageId.

    Can also help find a stageId list for a given rally.
    """

    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True, dbname=None):
        """Rallystage class. Builds on rally stages."""
        WRCRallyStages.__init__(self, sdbRallyId=sdbRallyId,
                                live=live, autoseed=autoseed,
                                nowarn=nowarn, dbname=dbname)

        if not nowarn:
            if not sdbRallyId:
                warnings.warn("sdbRallyId should really be set...")
            if not stageId:
                warnings.warn("stageId should really be set...")

        self.stageId = _jsInt(stageId)

        if autoseed:
            fetchData(self.sdbRallyId, stageId)

    def _checkStageId(self, sdbRallyId=None,
                      stageId=None, fallback='lastCompleted'):
        """Return a stage ID or lookup a current one."""
        self._checkRallyId(sdbRallyId)

        stageId = _jsInt(stageId)

        # One method for finding a stage ID - last completed
        if not stageId:
            if 'lastCompleted':
                stageId = _jsInt(self.lastCompletedStage())
            # What else? Most recent still running, else lastCompleted?
        return stageId
        # sdbRallyId = sdbRallyId or self.sdbRallyId
        # if not hasattr(self, 'sdbRallyId') or not self.sdbRallyId:
        #    self.activerally = WRCActiveRally()
        #    self.sdbRallyId = self.activerally.sdbRallyId
        #    self.name = self.activerally.name
        # return self.sdbRallyId


11:25: N803 argument name 'sdbRallyId' should be lowercase
29:10: N802 function name '_checkStageId' should be lowercase
29:30: N803 argument name 'sdbRallyId' should be lowercase
34:10: N806 variable 'stageId' in function should be lowercase
39:18: N806 variable 'stageId' in function should be lowercase


In [51]:
zz = WRCRallyStage()

# TO DO

Build in iterator to grab all stage data. overall, stagetimes and splittimes build on stage class.

In [52]:
class WRCOverall(WRCRallyStage):
    """Class for overall stage table."""

    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True, dbname=None):
        """Overall class. Builds on Rallystage class."""
        WRCRallyStage.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                               live=live, autoseed=autoseed,
                               nowarn=nowarn, dbname=dbname)

        self.overall = {}

        if stageId:
            self.fetchOverall(self.sdbRallyId, stageId)

        if autoseed:
            pass

    def fetchOverall(self, sdbRallyId=None, stageId=None):
        """Fetch the data from WRC API."""
        self._checkRallyId(sdbRallyId)
        self._checkStages(self.sdbRallyId)
        stageId = self._checkStageId(self.sdbRallyId, stageId)
        if stageId:
            self.overall[stageId] = getOverall(self.sdbRallyId, stageId)
            self.dbfy('overall', self.overall[stageId], ['stageId', 'entryId'])
    def __call__(self):
        """Make overall class callable. Returns dataframe."""
        return self.overall

4:25: N803 argument name 'sdbRallyId' should be lowercase
19:10: N802 function name 'fetchOverall' should be lowercase
19:29: N803 argument name 'sdbRallyId' should be lowercase
22:18: E231 missing whitespace after ','
24:18: E231 missing whitespace after ','
26:10: N806 variable 'stageId' in function should be lowercase
27:18: E231 missing whitespace after ','
35:5: E301 expected 1 blank line, found 0


In [53]:
zz = WRCOverall(stageId=1528, dbname=TESTDB)
#zz.fetchOverall()
#zz.rallyId

fetch...
1 None 1528
2 100 1528
upserting
2 100 1528
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S,1528
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S,1528
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S,1528
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S,1528
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S,1528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20768,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,82,287400,PT4M47.4S,0,PT0S,1528
82,20769,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,83,287400,PT4M47.4S,0,PT0S,1528
83,20770,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,84,287400,PT4M47.4S,0,PT0S,1528
84,20753,880800,PT14M40.8S,10000,PT10S,890800,PT14M50.8S,85,297400,PT4M57.4S,10000,PT10S,1528


2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '


upserting
5


In [54]:
zz = WRCOverall(stageId=1528, dbname=TESTDB)
zz.fetchOverall()
zz.dbtables()

fetch...
1 None 1528
2 100 1528
upserting
2 100 1528
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S,1528
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S,1528
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S,1528
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S,1528
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S,1528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20768,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,82,287400,PT4M47.4S,0,PT0S,1528
82,20769,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,83,287400,PT4M47.4S,0,PT0S,1528
83,20770,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,84,287400,PT4M47.4S,0,PT0S,1528
84,20753,880800,PT14M40.8S,10000,PT10S,890800,PT14M50.8S,85,297400,PT4M57.4S,10000,PT10S,1528


upserting
5
fetch...
1 None None
2 100 None
2 100 1526
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20684,11457600,PT3H10M57.6S,0,PT0S,11457600,PT3H10M57.6S,1,0,PT0S,0,PT0S,1526
1,20685,11470200,PT3H11M10.2S,0,PT0S,11470200,PT3H11M10.2S,2,12600,PT12.6S,12600,PT12.6S,1526
2,20686,11471900,PT3H11M11.9S,0,PT0S,11471900,PT3H11M11.9S,3,14300,PT14.3S,1700,PT1.7S,1526
3,20688,11646600,PT3H14M6.6S,0,PT0S,11646600,PT3H14M6.6S,4,189000,PT3M9S,174700,PT2M54.7S,1526
4,20690,11714800,PT3H15M14.8S,0,PT0S,11714800,PT3H15M14.8S,5,257200,PT4M17.2S,68200,PT1M8.2S,1526
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,20754,17370000,PT4H49M30S,120000,PT2M,17490000,PT4H51M30S,69,6032400,PT1H40M32.4S,31200,PT31.2S,1526
69,20762,17482400,PT4H51M22.4S,30000,PT30S,17512400,PT4H51M52.4S,70,6054800,PT1H40M54.8S,22400,PT22.4S,1526
70,20698,17680700,PT4H54M40.7S,0,PT0S,17680700,PT4H54M40.7S,71,6223100,PT1H43M43.1S,168300,PT2M48.3S,1526
71,20730,18208800,PT5H3M28.8S,10000,PT10S,18218800,PT5H3M38.8S,72,6761200,PT1H52M41.2S,538100,PT8M58.1S,1526


upserting
5


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


In [55]:
zz._checkStages(sdbRallyId)
display(zz.stages)

,stageId,eventId,number,name,distance,status,stageType,timingPrecision,locked,code,rallyid
0,1528,124,1,Malijai - Puimichel (Live TV),17.47,Interrupted,SpecialStage,Tenth,True,SS1,100
1,1538,124,2,Bayons - Bréziers,25.49,Completed,SpecialStage,Tenth,True,SS2,100
2,1533,124,3,Curbans - Venterol 1,20.02,Completed,SpecialStage,Tenth,True,SS3,100
3,1534,124,4,Saint-Clément - Freissinières 1,20.68,Completed,SpecialStage,Tenth,True,SS4,100
4,1535,124,5,Avançon - Notre-Dame-du-Laus 1,20.59,Completed,SpecialStage,Tenth,True,SS5,100
5,1539,124,6,Curbans - Venterol 2,20.02,Completed,SpecialStage,Tenth,True,SS6,100
6,1537,124,7,Saint-Clément - Freissinières 2,20.68,Completed,SpecialStage,Tenth,True,SS7,100
7,1536,124,8,Avançon - Notre-Dame-du-Laus 2,20.59,Completed,SpecialStage,Tenth,True,SS8,100
8,1532,124,9,St-Léger-les-Mélèzes - La Bâtie-Neuve 1,16.87,Completed,SpecialStage,Tenth,True,SS9,100
9,1531,124,10,La Bréole - Selonnet 1,20.73,Completed,SpecialStage,Tenth,True,SS10,100


In [56]:
class WRCStageTimes(WRCRallyStage):
    """Class for stage times table."""

    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, nowarn=True, dbname=None):
        """Stage times class builds on rally stage."""
        WRCRallyStage.__init__(self, sdbRallyId=sdbRallyId,
                               stageId=stageId, live=live,
                               autoseed=autoseed, nowarn=nowarn,
                               dbname=dbname)

        self.stagetimes={}

        if stageId:
            self.fetchStageTimes(self.sdbRallyId, stageId)

        if autoseed:
            pass

    def fetchStageTimes(self, sdbRallyId=None, stageId=None):
        """Fetch the data from WRC API."""
        stageId = _jsInt(stageId)
        self._checkRallyId(sdbRallyId)
        self._checkStages(self.sdbRallyId)
        self._checkStageId(self.sdbRallyId, stageId)
        if stageId:
            self.stagetimes[stageId] = getStageTimes(self.sdbRallyId, stageId)
            self.dbfy('stagetimes', self.stagetimes[stageId],
                      ['stageId', 'entryId'])

    def __call__(self):
        """Make stagetimes callable. Returns dataframe."""
        return self.stagetimes

4:25: N803 argument name 'sdbRallyId' should be lowercase
12:24: E225 missing whitespace around operator
20:10: N802 function name 'fetchStageTimes' should be lowercase
20:32: N803 argument name 'sdbRallyId' should be lowercase
22:10: N806 variable 'stageId' in function should be lowercase


In [57]:
zz = WRCStageTimes(stageId = 1528, dbname=TESTDB)
zz.fetchStageTimes()
zz.dbtables()

upserting
upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


1:27: E251 unexpected spaces around keyword / parameter equals
1:29: E251 unexpected spaces around keyword / parameter equals


In [58]:
## LINTED TO HERE

1:1: E266 too many leading '#' for block comment


In [59]:
class WRCSplitTimes(WRCRallyStage):
    """Class for SplitTimes stage table."""

    def __init__(self, sdbRallyId=None, stageId=None,
                 live=False, autoseed=False, dbname=None):
        """Split times class builds on rally stage."""
        WRCRallyStage.__init__(self, sdbRallyId=sdbRallyId,
                               stageId=stageId, live=live,
                               autoseed=autoseed, dbname=dbname)

        self.splitPoints = {}
        self.entrySplitPointTimes = {}
        self.splitPointTimes = {}

        if stageId:
            self.fetchSplitTimes(self.sdbRallyId, stageId)

        if autoseed:
            pass

    def fetchSplitTimes(self, sdbRallyId=None, stageId=None):
        """Get split times data from WRC API. Add to db if available."""
        stageId = _jsInt(stageId)
        self._checkRallyId(sdbRallyId)
        self._checkStages(self.sdbRallyId)
        self._checkStageId(self.sdbRallyId, stageId)
        if stageId:
            (self.splitPoints[stageId], self.entrySplitPointTimes[stageId],
             self.splitPointTimes[stageId]) = getSplitTimes(self.sdbRallyId, stageId)

            self.dbfy('splitPoints',
                      self.splitPoints[stageId],
                      ['splitPointId', 'stageId'])
            self.dbfy('entrySplitPointTimes',
                      self.entrySplitPointTimes[stageId],
                      ['stageId', 'entryId'])
            self.dbfy('splitPointTimes',
                      self.splitPointTimes[stageId],
                      ['splitPointTimeId', 'splitPointId', 'entryId', 'stageId'])

4:25: N803 argument name 'sdbRallyId' should be lowercase
21:10: N802 function name 'fetchSplitTimes' should be lowercase
21:32: N803 argument name 'sdbRallyId' should be lowercase
23:10: N806 variable 'stageId' in function should be lowercase
29:80: E501 line too long (85 > 79 characters)


In [60]:
zz = WRCSplitTimes(stageId = 1528, dbname=TESTDB)
zz.fetchSplitTimes()
zz.dbtables()


upserting
upserting
upserting
upserting


,type,name,tbl_name,rootpage,sql
0,table,season_categories,season_categories,2,CREATE TABLE [season_categories] (\n [id] IN...
1,table,championship_codes,championship_codes,3,CREATE TABLE [championship_codes] (\n [id] I...
2,table,event,event,4,CREATE TABLE [event] (\n [id] INTEGER PRIMAR...
3,table,days,days,5,CREATE TABLE [days] (\n [id] INTEGER PRIMARY...
4,table,channels,channels,6,CREATE TABLE [channels] (\n [id] INTEGER PRI...
5,table,championship,championship,19,CREATE TABLE [championship] (\n [championshi...
6,table,championshipRounds,championshipRounds,20,CREATE TABLE [championshipRounds] (\n [event...
7,table,championshipEntries,championshipEntries,24,CREATE TABLE [championshipEntries] (\n [cham...
8,table,championship_standings,championship_standings,26,CREATE TABLE [championship_standings] (\n [c...
9,table,round_results,round_results,28,CREATE TABLE [round_results] (\n [championsh...


1:27: E251 unexpected spaces around keyword / parameter equals
1:29: E251 unexpected spaces around keyword / parameter equals


TO DO - think about sqlite export.

In [61]:
def WRCdatagetter(func):
    """
    Run a passed in function then return a response.

    Originally included other logic...
    """
    def call(self):
        """Check attributes. If any missing, grab data as required."""
        (attrs, func2) = func(self)
        if not attrs:
            return None
        if isinstance(attrs, str):
            attrs = [attrs]
        # Don't call the function if all the attributes that would be set
        # by calling it are already set.
        if not all([_checkattr(self, a) for a in attrs]):
            func2()
        if len(attrs) > 1:
            return tuple(getattr(self, a) for a in attrs)
        return getattr(self, attrs[0])
    return call


# This class will contain everything about a single rally
class WRCEvent(WRCCars, WRCPenalties, WRCRetirements, WRCStartlist,
               WRCRally, WRCStagewinners, WRCOverall,
               WRCStageTimes, WRCSplitTimes):
    """
    Class for a rally event.

    Can be used to contain all the timing results data
    from a WRC rally weekend.
    """

    def __init__(self, sdbRallyId=None, stageId=None, live=False,
                 autoseed=False, slurp=False, dbname=None):
        """Build on classes for each page of API/WRC live timing website."""
        WRCCars.__init__(self, sdbRallyId=sdbRallyId, live=live,
                         autoseed=autoseed, dbname=dbname)
        WRCPenalties.__init__(self, sdbRallyId=sdbRallyId, live=live,
                              autoseed=autoseed, dbname=dbname)
        WRCRetirements.__init__(self, sdbRallyId=sdbRallyId, live=live,
                                autoseed=autoseed, dbname=dbname)
        WRCStartlist.__init__(self, startListId=None, live=live,
                              autoseed=autoseed, nowarn=True, dbname=dbname)
        WRCRally.__init__(self, sdbRallyId=sdbRallyId,
                          autoseed=autoseed, dbname=dbname)
        WRCStagewinners.__init__(self, sdbRallyId=sdbRallyId,
                                 autoseed=autoseed, dbname=dbname)
        WRCOverall.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                            autoseed=autoseed, dbname=dbname)
        WRCStageTimes.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                               autoseed=autoseed, dbname=dbname)
        WRCSplitTimes.__init__(self, sdbRallyId=sdbRallyId, stageId=stageId,
                               autoseed=autoseed, dbname=dbname)

        if slurp:
            self.rallyslurper()

    def getRally(self):
        """Get external rally details."""
        attrs = ['rally', 'eligibilities', 'groups']
        if not all([_checkattr(self, a) for a in attrs]):
            self.fetchRally()
        return tuple(getattr(self, a) for a in attrs)

    def getItinerary(self):
        """
        Get itinerary.

        If rally not known, use active rally.
        Also set a default startListId.
        """
        self.fetchItinerary()
        attrs = ['itinerary', 'legs', 'sections', 'controls', 'stages']
        return tuple(getattr(self, a) for a in attrs)

    def getStartlist(self, startListId=None):
        """
        Get startlist.

        If no startListId provided, try to find a default.
        """
        self.fetchStartList()
        attrs = ['startList', 'startListItems']
        return tuple(getattr(self, a) for a in attrs)

    def getPenalties(self):
        """Get penalties."""
        self.fetchPenalties()
        return self.penalties

    def getRetirements(self):
        """Get retirements."""
        self.fetchRetirements()
        return self.retirements

    def getStagewinners(self):
        """Get stagewinners."""
        self.fetchStagewinners()
        return self.stagewinners

    def getCars(self):
        """Get cars."""
        self.fetchCars()
        attrs = ['cars', 'classes']
        return tuple(getattr(self, a) for a in attrs)

    # TO DO - different decorator
    def getOverall(self, stageId=None):
        """Get Overall."""
        self._checkRallyId()
        self._checkStages()

        self.fetchOverall(self.sdbRallyId, stageId)

        return self.overall

    def getStageTimes(self, stageId=None):
        """Get StageTimes."""
        self._checkRallyId()
        self._checkStages()

        self.fetchStageTimes(self.sdbRallyId, stageId)
        return self.stagetimes

    def getSplitTimes(self, stageId=None):
        """Get SplitTimes."""
        self._checkRallyId()
        self._checkStages()

        self.fetchSplitTimes(self.sdbRallyId, stageId)

        return (self.splitPoints, self.entrySplitPointTimes,
                self.splitPointTimes)

    def rallyslurper(self):
        """Grab everything..."""
        self.getRally()
        self.getItinerary()
        self.getStartlist()
        self.getPenalties()
        self.getRetirements()
        self.getStagewinners()
        self.getCars()
        _stageIds = self.getStageIds()
        for ss in _stageIds:
            print(ss, _stageIds[ss]['stageId'])
            self.getOverall(_stageIds[ss]['stageId'])
            self.getStageTimes(_stageIds[ss]['stageId'])
            self.getSplitTimes(_stageIds[ss]['stageId'])

1:6: N802 function name 'WRCdatagetter' should be lowercase
35:25: N803 argument name 'sdbRallyId' should be lowercase
60:10: N802 function name 'getRally' should be lowercase
67:10: N802 function name 'getItinerary' should be lowercase
78:10: N802 function name 'getStartlist' should be lowercase
78:29: N803 argument name 'startListId' should be lowercase
88:10: N802 function name 'getPenalties' should be lowercase
93:10: N802 function name 'getRetirements' should be lowercase
98:10: N802 function name 'getStagewinners' should be lowercase
103:10: N802 function name 'getCars' should be lowercase
110:10: N802 function name 'getOverall' should be lowercase
110:27: N803 argument name 'stageId' should be lowercase
119:10: N802 function name 'getStageTimes' should be lowercase
119:30: N803 argument name 'stageId' should be lowercase
127:10: N802 function name 'getSplitTimes' should be lowercase
127:30: N803 argument name 'stageId' should be lowercase
146:10: N806 variable '_stageIds' in fun

In [62]:
!rm wrc2020bigtest1.db
zz = WRCEvent(dbname='wrc2020bigtest1.db')

zz.rallyslurper()

upserting
upserting
upserting
upserting
upserting
upserting
upserting
SS1 1528
fetch...
1 100 1528
2 100 1528
2 100 1528
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20685,593400,PT9M53.4S,0,PT0S,593400,PT9M53.4S,1,0,PT0S,0,PT0S,1528
1,20683,595200,PT9M55.2S,0,PT0S,595200,PT9M55.2S,2,1800,PT1.8S,1800,PT1.8S,1528
2,20686,595300,PT9M55.3S,0,PT0S,595300,PT9M55.3S,3,1900,PT1.9S,100,PT0.1S,1528
3,20684,599800,PT9M59.8S,0,PT0S,599800,PT9M59.8S,4,6400,PT6.4S,4500,PT4.5S,1528
4,20690,603600,PT10M3.6S,0,PT0S,603600,PT10M3.6S,5,10200,PT10.2S,3800,PT3.8S,1528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20768,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,82,287400,PT4M47.4S,0,PT0S,1528
82,20769,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,83,287400,PT4M47.4S,0,PT0S,1528
83,20770,880800,PT14M40.8S,0,PT0S,880800,PT14M40.8S,84,287400,PT4M47.4S,0,PT0S,1528
84,20753,880800,PT14M40.8S,10000,PT10S,890800,PT14M50.8S,85,297400,PT4M57.4S,10000,PT10S,1528


upserting
5
upserting
upserting
upserting
upserting
SS2 1538
fetch...
1 100 1538
2 100 1538
2 100 1538
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20684,1583500,PT26M23.5S,0,PT0S,1583500,PT26M23.5S,1,0,PT0S,0,PT0S,1538
1,20685,1602600,PT26M42.6S,0,PT0S,1602600,PT26M42.6S,2,19100,PT19.1S,19100,PT19.1S,1538
2,20683,1608600,PT26M48.6S,0,PT0S,1608600,PT26M48.6S,3,25100,PT25.1S,6000,PT6S,1538
3,20686,1608900,PT26M48.9S,0,PT0S,1608900,PT26M48.9S,4,25400,PT25.4S,300,PT0.3S,1538
4,20689,1634500,PT27M14.5S,0,PT0S,1634500,PT27M14.5S,5,51000,PT51S,25600,PT25.6S,1538
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,20711,2418900,PT40M18.9S,0,PT0S,2418900,PT40M18.9S,82,835400,PT13M55.4S,16300,PT16.3S,1538
82,20742,2436900,PT40M36.9S,0,PT0S,2436900,PT40M36.9S,83,853400,PT14M13.4S,18000,PT18S,1538
83,20767,2441700,PT40M41.7S,0,PT0S,2441700,PT40M41.7S,84,858200,PT14M18.2S,4800,PT4.8S,1538
84,20762,2559000,PT42M39S,0,PT0S,2559000,PT42M39S,85,975500,PT16M15.5S,117300,PT1M57.3S,1538


upserting
5
upserting
upserting
upserting
upserting
SS3 1533
fetch...
1 100 1533
2 100 1533
2 100 1533
3
4


...

upserting
5
upserting
upserting
upserting
upserting
SS15 1525
fetch...
1 100 1525
2 100 1525
2 100 1525
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20684,10878600,PT3H1M18.6S,0,PT0S,10878600,PT3H1M18.6S,1,0,PT0S,0,PT0S,1525
1,20686,10889700,PT3H1M29.7S,0,PT0S,10889700,PT3H1M29.7S,2,11100,PT11.1S,11100,PT11.1S,1525
2,20685,10891200,PT3H1M31.2S,0,PT0S,10891200,PT3H1M31.2S,3,12600,PT12.6S,1500,PT1.5S,1525
3,20688,11061400,PT3H4M21.4S,0,PT0S,11061400,PT3H4M21.4S,4,182800,PT3M2.8S,170200,PT2M50.2S,1525
4,20690,11123400,PT3H5M23.4S,0,PT0S,11123400,PT3H5M23.4S,5,244800,PT4M4.8S,62000,PT1M2S,1525
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,20754,16430900,PT4H33M50.9S,120000,PT2M,16550900,PT4H35M50.9S,69,5672300,PT1H34M32.3S,31200,PT31.2S,1525
69,20762,16543300,PT4H35M43.3S,30000,PT30S,16573300,PT4H36M13.3S,70,5694700,PT1H34M54.7S,22400,PT22.4S,1525
70,20698,16741600,PT4H39M1.6S,0,PT0S,16741600,PT4H39M1.6S,71,5863000,PT1H37M43S,168300,PT2M48.3S,1525
71,20730,17269700,PT4H47M49.7S,10000,PT10S,17279700,PT4H47M59.7S,72,6401100,PT1H46M41.1S,538100,PT8M58.1S,1525


upserting
5
upserting
upserting
upserting
upserting
SS16 1526
fetch...
1 100 1526
2 100 1526
2 100 1526
3
4


,entryId,stageTimeMs,stageTime,penaltyTimeMs,penaltyTime,totalTimeMs,totalTime,position,diffFirstMs,diffFirst,diffPrevMs,diffPrev,stageId
0,20684,11457600,PT3H10M57.6S,0,PT0S,11457600,PT3H10M57.6S,1,0,PT0S,0,PT0S,1526
1,20685,11470200,PT3H11M10.2S,0,PT0S,11470200,PT3H11M10.2S,2,12600,PT12.6S,12600,PT12.6S,1526
2,20686,11471900,PT3H11M11.9S,0,PT0S,11471900,PT3H11M11.9S,3,14300,PT14.3S,1700,PT1.7S,1526
3,20688,11646600,PT3H14M6.6S,0,PT0S,11646600,PT3H14M6.6S,4,189000,PT3M9S,174700,PT2M54.7S,1526
4,20690,11714800,PT3H15M14.8S,0,PT0S,11714800,PT3H15M14.8S,5,257200,PT4M17.2S,68200,PT1M8.2S,1526
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,20754,17370000,PT4H49M30S,120000,PT2M,17490000,PT4H51M30S,69,6032400,PT1H40M32.4S,31200,PT31.2S,1526
69,20762,17482400,PT4H51M22.4S,30000,PT30S,17512400,PT4H51M52.4S,70,6054800,PT1H40M54.8S,22400,PT22.4S,1526
70,20698,17680700,PT4H54M40.7S,0,PT0S,17680700,PT4H54M40.7S,71,6223100,PT1H43M43.1S,168300,PT2M48.3S,1526
71,20730,18208800,PT5H3M28.8S,10000,PT10S,18218800,PT5H3M38.8S,72,6761200,PT1H52M41.2S,538100,PT8M58.1S,1526


upserting
5
upserting
upserting
upserting
upserting


In [68]:
zz.pickle('wrc-montecarlo.pickle')

Pickled to wrc-montecarlo.pickle


In [64]:
zz.db_connect(dbname='wrc2020bigtest1.db')
zz.legs
int(zz.legs.loc[0, 'startListId'])

451

In [ ]:
#This class needs renaming...
#What does it actually represent? An event? A live event? A set of events?

# TO DO - this presumably is wrong if we call in in 2021?
class WRC2020(WRCActiveSeasonEvents, WRCEvent):
    """Class for WRC data scrape using 2020 API."""

    def __init__(self, sdbRallyId=None, live=False, autoseed=False):
        WRCActiveSeasonEvents.__init__(self, autoseed=autoseed)
        WRCEvent.__init__(self, sdbRallyId, live)
        
        self.live = live
        
        
    def getActiveSeasonEvents(self):
        """Get active (current) season events."""
        _current_season_events_attrs = ['current_season_events',
                                         'eventdays', 'eventchannel' ]
        if not any([hasattr(self,a) for a in _current_season_events_attrs]) or not _checkattr(self,'current_season_events'):
            self.fetchActiveSeasonEvents()
        return (self.current_season_events, self.eventdays, self.eventchannel)


    
    
    
    

In [ ]:
zz = WRC2020().getActiveSeasonEvents()
#zz

In [ ]:
#zz = WRCEvent(slurp=True)

In [ ]:
zz = WRCEvent(autoseed=True, sdbRallyId=100)
zz.getPenalties(sdbRallyId=100)

In [ ]:
wrc=WRC2020()
wrc.getActiveSeasonEvents()
wrc.activeseasonevents.current_season_events

In [ ]:
wrc.getStartlist()

In [ ]:
wrc.itinerary.sections

In [ ]:
itinerary, legs, sections, controls, stages = wrc.getItinerary()
startList,startListItems = getStartlist(startListId)
cars, classes = getCars(sdbRallyId)
#rally, eligibilities, groups = getRally(sdbRallyId)
#overall = getOverall(sdbRallyId, stageId)
#splitPoints, entrySplitPointTimes, splitPointTimes = getSplitTimes(sdbRallyId,stageId)
#stagetimes = getStageTimes(sdbRallyId,stageId)
#stagewinners = getStagewinners(sdbRallyId)
#penalties = getPenalties(sdbRallyId)
#retirements = getRetirements(sdbRallyId)
#championship = getChampionship()
#championship = getChampionshipStandingsLive()

In [ ]:
current_season_events, eventdays, eventchannel = wrc.getActiveSeasonEvents()

In [ ]:
event, days, channels = wrc.getActiveRally()


In [ ]:
event

In [ ]:
wrc.sdbRallyId